In [1]:
import torch
import torch.nn as nn
from PIL import Image
from torchvision import transforms
import sys
import time

# -----------------------------------
# 1. 모델 구조 정의 및 가중치 로드
# -----------------------------------
print("1. Loading the fine-tuned model structure and weights...")

# --- 설정값  ---
MODEL_REPO = "facebookresearch/dinov2"
MODEL_NAME = "dinov2_vits14"
# 클래스 수와 순서를 정확히 맞추기
NUM_CLASSES = 3 
CLASS_NAMES = ['downy','healthy', 'powdery']
DEVICE = torch.device("cpu")
print(f"--> Using device: {DEVICE}")
# --- 모델 구조 만들기  ---
try:
    # torch.hub를 이용해 DINOv2 모델 구조 로드
    model = torch.hub.load(MODEL_REPO, MODEL_NAME, pretrained=False) # pretrained=False로 설정
    
    # 마지막 분류기(머리)
    num_features = 384 # ViT-Small의 특징 벡터 크기
    model.head = nn.Linear(num_features, NUM_CLASSES)
    
    # --- 저장된 가중치 불러오기 ---
    model.load_state_dict(torch.load('dinov2_hub_finetuned_model.pth', map_location=DEVICE))
    
    model = model.to(DEVICE)
    model.eval() # 평가 모드
    print("--> Model loaded successfully.")

except Exception as e:
    print(f"\nERROR: Failed to load the model.")
    print(f"--> Ensure 'dinov2_hub_finetuned_model.pth' is in the same directory.")
    print(f"--> Original Error: {e}")
    sys.exit(1)

1. Loading the fine-tuned model structure and weights...
--> Using device: cpu


Using cache found in C:\Users\51100/.cache\torch\hub\facebookresearch_dinov2_main
C:\Users\51100/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\51100/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\51100/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


--> Model loaded successfully.


In [2]:
# -----------------------------------
# 2. 추론할 이미지 전처리
# -----------------------------------
print("\n2. Preparing the image for inference...")

# 이미지 전처리 파이프라인 (학습 validation 단계와 동일)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


2. Preparing the image for inference...


In [3]:
# -----------------------------------
# 3. 추론 실행 및 결과 확인
# -----------------------------------

def predict(image_path):
    # 시작 시간 기록
    start_time = time.time()
    """지정된 경로의 이미지에 대해 추론을 수행하고 결과를 출력합니다."""
    try:
        # 이미지 열기
        img = Image.open(image_path).convert("RGB")
        
        # 이미지 전처리 및 배치 차원 추가 (1, 3, 224, 224)
        img_tensor = preprocess(img).unsqueeze(0).to(DEVICE)
        
        print(f"\n3. Running inference on '{image_path}'...")
        
        # 기울기 계산 비활성화 (추론 시에는 필요 없음)
        with torch.no_grad():
            # 모델에 이미지 입력
            outputs = model(img_tensor)
            
            # Softmax를 적용하여 출력을 확률로 변환
            probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
            
            # 가장 높은 확률을 가진 클래스 찾기
            top_prob, top_catid = torch.max(probabilities, 0)
            
            # 종료 시간 기록
            end_time = time.time()
            
            # 결과 출력
            # 실행 시간 계산 (밀리초 단위)
            execution_time_ms = (end_time - start_time) * 1000
            print(f"코드 실행 시간: {execution_time_ms:.3f} ms")
            predicted_class = CLASS_NAMES[top_catid]
            confidence = top_prob.item()
            
            print("\n--- Inference Result ---")
            print(f"Predicted Class: {predicted_class}")
            print(f"Confidence: {confidence:.2%}")
            print("------------------------")
            return predicted_class, execution_time_ms
            
    except FileNotFoundError:
        print(f"ERROR: Image file not found at '{image_path}'")
    except Exception as e:
        print(f"ERROR: An error occurred during inference: {e}")

In [5]:
import os
import shutil

def get_test_list(target_dir):
    try:
        all_files = os.listdir(target_dir)
        dir = []
        for file in all_files:
            dir.append(target_dir + file)
        return dir
        
    except FileNotFoundError:
        print(f"❌ 오류: 원본 폴더 '{target_dir}'을(를) 찾을 수 없습니다.")
    except Exception as e:
        print(f"❌ 알 수 없는 오류가 발생했습니다: {e}")

In [6]:
healthy = get_test_list('C:/blooming_AI/classification_dataset/test/healthy/')
powdery = get_test_list('C:/blooming_AI/classification_dataset/test/powdery/')
downy = get_test_list('C:/blooming_AI/classification_dataset/test/downy/')

In [8]:
import pandas as pd
import numpy as np
result = pd.DataFrame(np.zeros((3,3)), ['healthy', 'powdery', 'downy'], ['healthy', 'powdery', 'downy'])

In [9]:
t = 0
# 정상 잎 테스트
for image in healthy:
    cla, ti = predict(image)
    result.at['healthy', cla] += 1
    t += ti
print(f"정상 {result.at['healthy', 'healthy']}개 | 흰가루병 {result.at['healthy', 'powdery']}개 | 노균병 {result.at['healthy','downy']}개 |")
print(f"평균추론시간 = {(t / len(healthy)):.3f}ms")


3. Running inference on 'C:/blooming_AI/classification_dataset/test/healthy/427493_20201001_4_0_0_3_2_13_0_3.jpg'...
코드 실행 시간: 161.347 ms

--- Inference Result ---
Predicted Class: healthy
Confidence: 69.56%
------------------------

3. Running inference on 'C:/blooming_AI/classification_dataset/test/healthy/427496_20201001_4_0_0_3_2_13_0_6.jpg'...
코드 실행 시간: 179.168 ms

--- Inference Result ---
Predicted Class: healthy
Confidence: 99.98%
------------------------

3. Running inference on 'C:/blooming_AI/classification_dataset/test/healthy/427544_20201001_4_0_0_3_2_13_0_54.jpg'...
코드 실행 시간: 172.065 ms

--- Inference Result ---
Predicted Class: healthy
Confidence: 99.92%
------------------------

3. Running inference on 'C:/blooming_AI/classification_dataset/test/healthy/427548_20201001_4_0_0_3_2_13_0_58.jpg'...
코드 실행 시간: 180.510 ms

--- Inference Result ---
Predicted Class: healthy
Confidence: 99.60%
------------------------

3. Running inference on 'C:/blooming_AI/classification_datase

In [10]:
t2 = 0
# 흰가루병 테스트
for image in powdery:
    cla, ti = predict(image)
    result.at['powdery', cla] += 1
    t2 += ti
print(f"정상 {result.at['powdery', 'healthy']}개 | 흰가루병 {result.at['powdery', 'powdery']}개 | 노균병 {result.at['powdery','downy']}개 |")
print(f"평균추론시간 = {(t2 / len(powdery)):.3f}ms")


3. Running inference on 'C:/blooming_AI/classification_dataset/test/powdery/296174_20210910_4_1_a4_3_2_12_1_0.jpg'...
코드 실행 시간: 154.642 ms

--- Inference Result ---
Predicted Class: powdery
Confidence: 100.00%
------------------------

3. Running inference on 'C:/blooming_AI/classification_dataset/test/powdery/296178_20210910_4_1_a4_3_2_11_0_0.jpg'...
코드 실행 시간: 182.812 ms

--- Inference Result ---
Predicted Class: powdery
Confidence: 100.00%
------------------------

3. Running inference on 'C:/blooming_AI/classification_dataset/test/powdery/296188_20210910_4_1_a4_3_2_12_1_2.jpg'...
코드 실행 시간: 145.826 ms

--- Inference Result ---
Predicted Class: powdery
Confidence: 100.00%
------------------------

3. Running inference on 'C:/blooming_AI/classification_dataset/test/powdery/296189_20210910_4_1_a4_3_2_12_1_3.jpg'...
코드 실행 시간: 157.225 ms

--- Inference Result ---
Predicted Class: powdery
Confidence: 100.00%
------------------------

3. Running inference on 'C:/blooming_AI/classification_

In [11]:
t3 = 0
# 노균병 테스트
for image in downy:
    cla, ti = predict(image)
    result.at['downy',cla] += 1
    t3 += ti
print(f"정상 {result.at['downy','healthy']}개 | 흰가루병 {result.at['downy','powdery']}개 | 노균병 {result.at['downy','downy']}개 |")
print(f"평균추론시간 = {(t3 / len(downy)):.3f}ms")


3. Running inference on 'C:/blooming_AI/classification_dataset/test/downy/470538_20211014_4_1_a3_3_2_13_1_11.jpg'...
코드 실행 시간: 159.278 ms

--- Inference Result ---
Predicted Class: downy
Confidence: 100.00%
------------------------

3. Running inference on 'C:/blooming_AI/classification_dataset/test/downy/470543_20211014_4_1_a3_3_2_13_1_16.jpg'...
코드 실행 시간: 153.359 ms

--- Inference Result ---
Predicted Class: downy
Confidence: 100.00%
------------------------

3. Running inference on 'C:/blooming_AI/classification_dataset/test/downy/470546_20211014_4_1_a3_3_2_13_1_19.jpg'...
코드 실행 시간: 153.444 ms

--- Inference Result ---
Predicted Class: downy
Confidence: 95.07%
------------------------

3. Running inference on 'C:/blooming_AI/classification_dataset/test/downy/470547_20211014_4_1_a3_3_2_13_1_20.jpg'...
코드 실행 시간: 161.857 ms

--- Inference Result ---
Predicted Class: downy
Confidence: 94.04%
------------------------

3. Running inference on 'C:/blooming_AI/classification_dataset/test/d

In [15]:
print(f"최종 평균 추론시간 = {(t+t2+t3) / (len(healthy) + len(powdery) + len(downy)):.3f}ms (연산 장치: 13th Gen Intel(R) Core(TM) i7-13700K)")
# 전체 정확도 (Accuracy) 계산
accuracy = np.diag(result).sum() / result.values.sum()
print(f"## 전체 정확도 (Overall Accuracy)\n- {accuracy:.4f}\n")
classes = ['healthy', 'powdery', 'downy']
# 각 클래스별 지표를 저장할 DataFrame 생성
metrics_df = pd.DataFrame(index=classes, columns=['Precision', 'Recall', 'F1-Score'])

# 각 클래스에 대해 Precision, Recall, F1-Score 계산
for cls in classes:
    tp = result.loc[cls, cls]
    fp = result[cls].sum() - tp
    fn = result.loc[cls].sum() - tp
    
    # Precision (정밀도)
    precision = tp / (tp + fp)
    
    # Recall (재현율)
    recall = tp / (tp + fn)
    
    # F1-Score
    f1_score = 2 * (precision * recall) / (precision + recall)
    
    metrics_df.loc[cls] = [precision, recall, f1_score]

print("## 클래스별 평가 지표\n")
print(metrics_df)

# Macro Average와 Weighted Average 계산
macro_avg = metrics_df.mean()
support = result.sum(axis=1) # 각 클래스의 실제 샘플 수
weighted_avg = (metrics_df.T * support).T.sum() / support.sum()

print("\n## 평균 지표\n")
print(f"- Macro Average:\n{macro_avg.to_string()}\n")
print(f"- Weighted Average:\n{weighted_avg.to_string()}")

최종 평균 추론시간 = 155.221ms (연산 장치: 13th Gen Intel(R) Core(TM) i7-13700K)
## 전체 정확도 (Overall Accuracy)
- 0.9945

## 클래스별 평가 지표

        Precision    Recall  F1-Score
healthy  0.991107  0.993561  0.992332
powdery  0.999044  0.999522  0.999282
downy    0.992908  0.989082  0.990991

## 평균 지표

- Macro Average:
Precision    0.994353
Recall       0.994055
F1-Score     0.994202

- Weighted Average:
Precision    0.994529
Recall       0.994529
F1-Score     0.994527


In [16]:
result

,healthy,powdery,downy
healthy,2006.0,2.0,11.0
powdery,1.0,2089.0,0.0
downy,17.0,0.0,1540.0
